In [ ]:
"""
STEP File Surface Analyzer

This module provides functions to analyze STEP files and extract information
about surface types and their quantities.
"""

import FreeCAD
import FreeCADGui as Gui
import Part
from collections import defaultdict
from typing import Dict, List, Tuple
from pathlib import Path
import math, logging

: 

In [4]:
class Countersink:
    """Class representing a detected countersink (cylindrical or conical)"""
    def __init__(self, sink_type: str, diameter: float, depth: float, angle: float = None):
        self.type = sink_type  # "cylindrical" or "conical"
        self.diameter = diameter
        self.depth = depth
        self.angle = angle  # Only for conical

    def __repr__(self):
        if self.type == "conical":
            return (f"Countersink(type='conical', diameter={self.diameter:.2f}, "
                    f"depth={self.depth:.2f}, angle={self.angle:.1f}°)")
        else:
            return (f"Countersink(type='cylindrical', diameter={self.diameter:.2f}, "
                    f"depth={self.depth:.2f})")


class Hole:
    """Enhanced Hole class with countersink support"""
    def __init__(self, face : Part.Face, hole_type: str, diameter: float, depth: float, 
                 position, axis, countersinks: List[Countersink] = None):
        self.face = face
        self.hole_type = hole_type
        self.diameter = diameter
        self.depth = depth
        self.position = position
        self.axis = axis
        self.countersinks = countersinks or []
        
    def __repr__(self):
        base = (f"Hole(type='{self.hole_type}', diameter={self.diameter:.2f}, "
                f"depth={self.depth:.2f}, countersinks={len(self.countersinks)})"
                f'Face={self.face}')
        return base


def detect_countersink(face, shape, tolerance=1e-3):
    """
    Detects a countersink (cylindrical or conical) at the ends of a hole face.
    Returns a tuple: (list of Countersink objects, set of faces used for countersinks)
    """
    sinks = []
    countersink_faces = set()  # Sammle alle Flächen, die als Senkung verwendet werden
    hole_radius = face.Surface.Radius
    
    for edge in face.Edges:
        # Suche angrenzende Flächen an den Kreisflächen der Bohrung
        if is_circular_edge(edge, tolerance):
            for neighbor in get_face_neighbors(shape, face):
                if not hasattr(neighbor, "Surface") or not hasattr(neighbor.Surface, "TypeId"):
                    continue
                    
                surf = neighbor.Surface
                
                # Zylindrische Senkung (Cylindrical Countersink)
                if surf.TypeId == 'Part::GeomCylinder':
                    neighbor_radius = surf.Radius
                    # Prüfe, ob Durchmesser größer als Bohrung
                    if neighbor_radius > hole_radius + tolerance:
                        depth = get_hole_depth(neighbor, shape, "through")
                        sinks.append(Countersink(
                            sink_type="cylindrical",
                            diameter=neighbor_radius * 2,
                            depth=depth
                        ))
                        countersink_faces.add(id(neighbor))  # Markiere diese Fläche
                
                # Konische Senkung (Conical Countersink)
                elif surf.TypeId == 'Part::GeomCone':
                    # Hole größeren Radius (Basis oder Apex)
                    radius1 = surf.Radius1 if hasattr(surf, 'Radius1') else surf.Radius
                    radius2 = surf.Radius2 if hasattr(surf, 'Radius2') else 0
                    larger_radius = max(radius1, radius2)
                    
                    # Prüfe, ob größerer Radius größer als Bohrung
                    if larger_radius > hole_radius + tolerance:
                        depth = get_hole_depth(neighbor, shape, "through")
                        
                        # Berechne Senk-Winkel
                        angle = None
                        if hasattr(surf, "SemiAngle"):
                            angle = surf.SemiAngle * 180.0 / math.pi * 2  # Voller Öffnungswinkel
                        
                        sinks.append(Countersink(
                            sink_type="conical",
                            diameter=larger_radius * 2,
                            depth=depth,
                            angle=angle
                        ))
                        countersink_faces.add(id(neighbor))  # Markiere diese Fläche
    
    return sinks, countersink_faces


def classify_hole_with_countersink(face, shape, countersinks: List[Countersink], 
                                   hole_type: str) -> str:
    """
    Klassifiziert ein Loch basierend auf vorhandenen Countersinks.
    
    Returns:
        - "countersunk_hole": Loch mit konischer Senkung (für Senkkopfschrauben)
        - "cylinderhead_hole": Loch mit zylindrischer Senkung (für Zylinderkopfschrauben)
        - Original hole_type: Falls keine Senkung vorhanden
    """
    if not countersinks:
        return hole_type
    
    # Prüfe auf konische Senkung (Countersunk Hole)
    has_conical = any(cs.type == "conical" for cs in countersinks)
    if has_conical:
        return "countersunk_hole"
    
    # Prüfe auf zylindrische Senkung (Cylinderhead Hole)
    has_cylindrical = any(cs.type == "cylindrical" for cs in countersinks)
    if has_cylindrical:
        return "cylinderhead_hole"
    
    return hole_type


def detect_holes(shape) -> List[Hole]:
    """
    Detect all holes in a shape, including countersinks.
    Automatically classifies holes as countersunk_hole or cylinderhead_hole.
    Prevents countersink faces from being detected as separate holes.
    """
    holes = []
    excluded_faces = set()  # Sammle alle Flächen, die als Senkungen identifiziert wurden

    # Erster Durchlauf: Erkenne alle Bohrungen und ihre Senkungen
    potential_holes = []
    for face in shape.Faces:
        if is_cylindrical_face(face) and is_hole_feature(face, shape):
            # Get cylinder information
            radius, center, axis = get_cylinder_info(face)
            diameter = radius * 2

            # Classify the hole (basic classification)
            hole_type = classify_hole(face, shape)

            # Get depth
            depth = get_hole_depth(face, shape, hole_type)

            # Detect countersinks at hole ends
            countersinks, countersink_face_ids = detect_countersink(face, shape)
            
            # Sammle alle Senkungsflächen
            excluded_faces.update(countersink_face_ids)

            # Re-classify based on countersinks
            final_hole_type = classify_hole_with_countersink(
                face, shape, countersinks, hole_type
            )

            potential_holes.append({
                'face': face,
                'hole_type': final_hole_type,
                'diameter': diameter,
                'depth': depth,
                'center': center,
                'axis': axis,
                'countersinks': countersinks
            })

    # Zweiter Durchlauf: Filtere Bohrungen, deren Flächen als Senkungen markiert sind
    for hole_data in potential_holes:
        face_id = id(hole_data['face'])
        
        # Überspringe diese Fläche, wenn sie als Senkung einer anderen Bohrung identifiziert wurde
        if face_id in excluded_faces:
            continue
        
        # Create hole object with countersinks
        hole = Hole(
            face = hole_data['face'],
            hole_type=hole_data['hole_type'],
            diameter=hole_data['diameter'],
            depth=hole_data['depth'],
            position=hole_data['center'],
            axis=hole_data['axis'],
            countersinks=hole_data['countersinks']
        )

        holes.append(hole)

    return holes


def print_hole_info(holes: List[Hole]):
    """Hilfsfunktion zum Ausgeben der erkannten Löcher"""
    print(f"\n{'='*50}")
    print(f"Found {len(holes)} hole(s)")
    print(f"{'='*50}")
    
    for i, hole in enumerate(holes, 1):
        print(f"\n--- Hole {i} ---")
        print(f"Type: {hole.hole_type}")
        print(f"Diameter: {hole.diameter:.2f} mm")
        print(f"Depth: {hole.depth:.2f} mm")
        print(f"Position: {hole.position}")
        print(f"Axis: {hole.axis}")
        
        if hole.countersinks:
            print(f"Countersinks: {len(hole.countersinks)}")
            for j, cs in enumerate(hole.countersinks, 1):
                print(f"  {j}. {cs}")



def is_cylindrical_face(face) -> bool:
    """
    Check if a face is cylindrical.

    Args:
        face: FreeCAD face object

    Returns:
        True if the face is cylindrical
    """
    try:
        return face.Surface.TypeId == 'Part::GeomCylinder'
    except:
        return False


def get_cylinder_info(face) -> Tuple[float, Tuple[float, float, float], Tuple[float, float, float]]:
    """
    Extract cylinder information from a cylindrical face.

    Args:
        face: Cylindrical FreeCAD face

    Returns:
        Tuple of (radius, center_position, axis_direction)
    """
    surface = face.Surface
    radius = surface.Radius

    # Get cylinder axis
    axis = surface.Axis
    axis_tuple = (axis.x, axis.y, axis.z)

    # Get a point on the cylinder axis
    center = surface.Center
    center_tuple = (center.x, center.y, center.z)

    return radius, center_tuple, axis_tuple


def get_face_neighbors(shape, face) -> List:
    """
    Get neighboring faces that share edges with the given face.

    Args:
        shape: FreeCAD shape object
        face: Face to find neighbors for

    Returns:
        List of neighboring faces
    """
    neighbors = []
    face_edges = set(face.Edges)

    for other_face in shape.Faces:
        if other_face.isEqual(face):
            continue

        other_edges = set(other_face.Edges)
        # Check if faces share any edges
        if face_edges & other_edges:
            neighbors.append(other_face)

    return neighbors


def is_circular_edge(edge, tolerance=1e-3) -> bool:
    """
    Check if an edge is circular.

    Args:
        edge: FreeCAD edge object
        tolerance: Tolerance for comparisons

    Returns:
        True if the edge is circular
    """
    try:
        curve = edge.Curve
        # Check if it's a circle
        return curve.TypeId == 'Part::GeomCircle' and edge.isClosed()
    except:
        return False


def has_closed_circular_edges(face, tolerance=1e-3) -> bool:
    """
    Check if a cylindrical face has closed circular edges at its ends.
    This is a key characteristic of a drilled hole.

    Args:
        face: Cylindrical face to check
        tolerance: Tolerance for comparisons

    Returns:
        True if the face has at least one closed circular edge
    """
    if not is_cylindrical_face(face):
        return False

    surface = face.Surface
    radius = surface.Radius

    # Count closed circular edges with matching radius
    circular_edges = 0

    for edge in face.Edges:
        if is_circular_edge(edge, tolerance):
            # Check if the radius matches the cylinder radius
            curve = edge.Curve
            edge_radius = curve.Radius

            if abs(edge_radius - radius) < tolerance:
                circular_edges += 1

    # A proper hole should have exactly two circular edges 
    return circular_edges >= 1


def is_empty_inside(face, shape, tolerance=1e-3, num_samples=5) -> bool:
    """
    Check if there is no material inside the cylindrical feature.
    This distinguishes holes from other cylindrical features like fillets.

    We sample points along the cylinder axis at a smaller radius and check
    if they are inside the solid material. For a true hole, these points
    should be outside the solid (in the empty space).

    Args:
        face: Cylindrical face to check
        shape: Parent shape
        tolerance: Tolerance for comparisons
        num_samples: Number of sample points to check along the axis

    Returns:
        True if there is no material inside the cylinder
    """
    if not is_cylindrical_face(face):
        return False

    try:
        surface = face.Surface
        radius = surface.Radius
        axis = surface.Axis
        axis_center = surface.Center

        # Normalize axis direction
        axis_length = math.sqrt(axis.x**2 + axis.y**2 + axis.z**2)
        if axis_length < tolerance:
            return False

        axis_normalized = FreeCAD.Vector(
            axis.x / axis_length,
            axis.y / axis_length,
            axis.z / axis_length
        )

        # Get parameter range to determine the extent along the axis
        v_start = face.ParameterRange[2]
        v_end = face.ParameterRange[3]

        # Calculate sample points along the cylinder axis
        # We sample at a smaller radius (e.g., 50% of cylinder radius)
        # to be well inside the hole if it exists
        sample_radius = radius * 0.5

        # Sample along the axis at different heights
        for i in range(num_samples):
            t = i / max(num_samples - 1, 1)
            v = v_start + t * (v_end - v_start)

            # Get a point on the cylinder surface
            u_mid = (face.ParameterRange[0] + face.ParameterRange[1]) / 2
            surface_point = face.valueAt(u_mid, v)

            # Calculate a point on the axis at this height
            # Project surface_point onto the axis
            to_surface = surface_point.sub(axis_center)
            along_axis = to_surface.dot(axis_normalized)
            axis_point = axis_center.add(axis_normalized.multiply(along_axis))

            # Now create a sample point at sample_radius from the axis
            # in a perpendicular direction
            radial_direction = surface_point.sub(axis_point)
            radial_length = radial_direction.Length

            if radial_length < tolerance:
                # We're on the axis, pick an arbitrary perpendicular direction
                sample_point = axis_point
            else:
                radial_normalized = radial_direction.multiply(1.0 / radial_length)
                sample_point = axis_point.add(radial_normalized.multiply(sample_radius))

            # Check if this sample point is inside the solid
            # For a hole, it should NOT be inside the solid (it's in empty space)
            is_inside = shape.isInside(sample_point, tolerance, True)

            if is_inside:
                # There is material at this location, so it's not a hole
                return False

        # All sample points are outside the solid, confirming it's empty inside
        return True

    except Exception as e:
        # If we can't determine, default to False
        import traceback
        print(f"Error in is_empty_inside: {e}")
        traceback.print_exc()
        return False


def is_hole_feature(face, shape, tolerance=1e-3) -> bool:
    """
    Determine if a cylindrical face represents a hole (negative feature).

    A hole must meet this criteria:
    1. It must be a cylindrical surface
    2. It must have closed circular edges (defining the hole opening)
    3. Either it's empty inside OR it's part of a coaxial cylinder group (stepped hole)
    4. For coaxial groups, we want the LARGEST cylinder (main hole diameter)

    Args:
        face: Cylindrical face to check
        shape: Parent shape
        tolerance: Tolerance for comparisons

    Returns:
        True if the face represents a hole
    """
    if not is_cylindrical_face(face):
        return False

    # Check 1: Must have closed circular edges
    if not has_closed_circular_edges(face, tolerance):
        return False
    
    # Check 2: Must be empty inside (no material in the cylinder)
    # This is the key characteristic of a drilled hole
    if not is_empty_inside(face, shape, tolerance):
        return False

    # surface = face.Surface
    # radius = surface.Radius
    # axis = surface.Axis
    # center = surface.Center

    # # Check 3: Reject features with coaxial cylinders (grooves/pockets, not holes)
    # # If there are coaxial cylinders, it means there's material inside -> NOT a hole
    # # Example: A groove for a seal/gasket has an outer and inner cylinder
    # coaxial_cylinders = []

    # for other_face in shape.Faces:
    #     if other_face.isEqual(face):
    #         continue  # Skip self

    #     if is_cylindrical_face(other_face) and has_closed_circular_edges(other_face, tolerance):
    #         other_surface = other_face.Surface
    #         other_radius = other_surface.Radius
    #         other_axis = other_surface.Axis
    #         other_center = other_surface.Center

    #         # Check if cylinders are coaxial (same axis and center)
    #         axis_parallel = abs(axis.dot(other_axis)) > 0.99
    #         centers_close = center.distanceToPoint(other_center) < tolerance * 10

    #         if axis_parallel and centers_close:
    #             coaxial_cylinders.append((other_face, other_radius))

    # # If we found coaxial cylinders, this is a groove/pocket, NOT a hole
    # if len(coaxial_cylinders) > 0:
    #     return False

    return True


def classify_hole(face, shape, tolerance=1e-3) -> str:
    """
    Classify a hole as through or blind.

    Strategy:
    Compare the hole depth to the part dimension along the hole axis.
    - If hole depth >= ~80% of part dimension → through hole
    - If hole depth < ~80% of part dimension → blind hole

    Args:
        face: Cylindrical face representing the hole
        shape: Parent shape
        tolerance: Tolerance for comparisons

    Returns:
        "through" or "blind"
    """
    surface = face.Surface # is a cylindrical face
    axis_norm = face.Surface.Axis

    # get the hole depth by the distance of the circle centers of the end circles of the cylinder face
    circles = [edge for edge in face.Edges if edge.Curve.TypeId == 'Part::GeomCircle']
    hole_depth = round((circles[0].CenterOfGravity-circles[1].CenterOfGravity).Length)

    # Get the shape bounding box
    shape_bbox = shape.BoundBox



    # Calculate part dimension along the hole axis
    # Project the bounding box onto the hole axis
    dx = shape_bbox.XLength if abs(axis_norm.x) > 0.5 else 0
    dy = shape_bbox.YLength if abs(axis_norm.y) > 0.5 else 0
    dz = shape_bbox.ZLength if abs(axis_norm.z) > 0.5 else 0

    part_dimension = max(dx, dy, dz)

    if part_dimension < tolerance:
        return "blind"

    # Calculate ratio
    ratio = hole_depth / part_dimension

    # Classification:
    # - ratio >= 0.8 → through hole (goes most/all of the way through)
    # - ratio < 0.8 → blind hole (only goes partway)
    if ratio >= 0.8:
        return "through"
    else:
        return "blind"


def get_hole_depth(face, shape, hole_type: str) -> float:
    """
    Calculate the depth of a hole as the length of the face bounding box
    projected onto the cylinder axis.
    """
    try:
        bbox = face.BoundBox

        # Get cylinder axis
        surface = face.Surface
        axis = surface.Axis

        # Compute axis length and normalized axis
        axis_length = math.sqrt(axis.x**2 + axis.y**2 + axis.z**2)
        if axis_length <= 1e-9:
            # Fallback to diagonal length if axis is degenerate
            return bbox.DiagonalLength

        axis_norm = FreeCAD.Vector(axis.x / axis_length,
                                   axis.y / axis_length,
                                   axis.z / axis_length)

        # Build the 8 corners of the bounding box
        xs = (bbox.XMin, bbox.XMax)
        ys = (bbox.YMin, bbox.YMax)
        zs = (bbox.ZMin, bbox.ZMax)

        corners = [FreeCAD.Vector(x, y, z) for x in xs for y in ys for z in zs]

        # Project all corners onto the axis and take the span
        projections = [corner.dot(axis_norm) for corner in corners]
        depth = max(projections) - min(projections)

        # Defensive: ensure non-negative
        if depth < 0:
            depth = abs(depth)

        # If depth is extremely small, fallback
        if depth <= 1e-9:
            return bbox.DiagonalLength

        return depth

    except Exception:
        # On any error, return a safe fallback
        return face.BoundBox.DiagonalLength


# def detect_holes(shape) -> List[Hole]:
#     """
#     Detect all holes in a shape.

#     Args:
#         shape: FreeCAD shape object

#     Returns:
#         List of Hole objects
#     """
#     holes = []

#     for face in shape.Faces:
#         if is_cylindrical_face(face) and is_hole_feature(face, shape):
#             # Get cylinder information
#             radius, center, axis = get_cylinder_info(face)
#             diameter = radius * 2

#             # Classify the hole
#             hole_type = classify_hole(face, shape)

#             # Get depth
#             depth = get_hole_depth(face, shape, hole_type)

#             # Create hole object
#             hole = Hole(
#                 hole_type=hole_type,
#                 diameter=diameter,
#                 depth=depth,
#                 position=center,
#                 axis=axis
#             )

#             holes.append(hole)

    # return holes


def analyze_machining_features(doc: FreeCAD.Document) -> Dict[str, List[Hole]]:
    """
    Analyze all machining features in a FreeCAD document.

    Args:
        doc: FreeCAD document

    Returns:
        Dictionary with feature types as keys and lists of features as values
    """
    all_holes = []

    for obj in doc.Objects:
        if hasattr(obj, 'Shape'):
            shape = obj.Shape
            holes = detect_holes(shape)
            all_holes.extend(holes)

    # Organize by type
    features = {
        'through_holes': [h for h in all_holes if h.type == 'through'],
        'blind_holes': [h for h in all_holes if h.type == 'blind']
    }

    return features


def get_machining_statistics(features: Dict[str, List[Hole]]) -> Dict[str, int]:
    """
    Get statistics about machining features.

    Args:
        features: Dictionary of features from analyze_machining_features

    Returns:
        Dictionary with feature counts
    """
    stats = {
        'total_holes': len(features['through_holes']) + len(features['blind_holes']),
        'through_holes': len(features['through_holes']),
        'blind_holes': len(features['blind_holes'])
    }

    return stats



def analyze_step_file_features(file_path: str) -> Tuple[Dict[str, List[Hole]], Dict[str, int]]:
    """
    Main function to analyze machining features in a STEP file.

    Args:
        file_path: Path to the STEP file

    Returns:
        Tuple of (features_dict, statistics_dict)
    """
    from step_analyzer import load_step_file

    doc = load_step_file(file_path)
    features = analyze_machining_features(doc)
    stats = get_machining_statistics(features)

    FreeCAD.closeDocument(doc.Name)

    return features, stats


from typing import List, Optional, Set
import math


def detect_countersink(face, shape, tolerance=1e-3):
    """
    Detects a countersink (cylindrical or conical) at the ends of a hole face.
    Returns a tuple: (list of Countersink objects, set of faces used for countersinks)
    """
    sinks = []
    countersink_faces = set()  # Sammle alle Flächen, die als Senkung verwendet werden
    hole_radius = face.Surface.Radius
    
    for edge in face.Edges:
        # Suche angrenzende Flächen an den Kreisflächen der Bohrung
        if is_circular_edge(edge, tolerance):
            for neighbor in get_face_neighbors(shape, face):
                if not hasattr(neighbor, "Surface") or not hasattr(neighbor.Surface, "TypeId"):
                    continue
                    
                surf = neighbor.Surface
                
                # Zylindrische Senkung (Cylindrical Countersink)
                if surf.TypeId == 'Part::GeomCylinder':
                    neighbor_radius = surf.Radius
                    # Prüfe, ob Durchmesser größer als Bohrung
                    if neighbor_radius > hole_radius + tolerance:
                        depth = get_hole_depth(neighbor, shape, "through")
                        sinks.append(Countersink(
                            sink_type="cylindrical",
                            diameter=neighbor_radius * 2,
                            depth=depth
                        ))
                        countersink_faces.add(id(neighbor))  # Markiere diese Fläche
                
                # Konische Senkung (Conical Countersink)
                elif surf.TypeId == 'Part::GeomCone':
                    # Hole größeren Radius (Basis oder Apex)
                    radius1 = surf.Radius1 if hasattr(surf, 'Radius1') else surf.Radius
                    radius2 = surf.Radius2 if hasattr(surf, 'Radius2') else 0
                    larger_radius = max(radius1, radius2)
                    
                    # Prüfe, ob größerer Radius größer als Bohrung
                    if larger_radius > hole_radius + tolerance:
                        depth = get_hole_depth(neighbor, shape, "through")
                        
                        # Berechne Senk-Winkel
                        angle = None
                        if hasattr(surf, "SemiAngle"):
                            angle = surf.SemiAngle * 180.0 / math.pi * 2  # Voller Öffnungswinkel
                        
                        sinks.append(Countersink(
                            sink_type="conical",
                            diameter=larger_radius * 2,
                            depth=depth,
                            angle=angle
                        ))
                        countersink_faces.add(id(neighbor))  # Markiere diese Fläche
    
    return sinks, countersink_faces


def classify_hole_with_countersink(face, shape, countersinks: List[Countersink], 
                                   hole_type: str) -> str:
    """
    Klassifiziert ein Loch basierend auf vorhandenen Countersinks.
    
    Returns:
        - "countersunk_hole": Loch mit konischer Senkung (für Senkkopfschrauben)
        - "cylinderhead_hole": Loch mit zylindrischer Senkung (für Zylinderkopfschrauben)
        - Original hole_type: Falls keine Senkung vorhanden
    """
    if not countersinks:
        return hole_type
    
    # Prüfe auf konische Senkung (Countersunk Hole)
    has_conical = any(cs.type == "conical" for cs in countersinks)
    if has_conical:
        return "countersunk_hole"
    
    # Prüfe auf zylindrische Senkung (Cylinderhead Hole)
    has_cylindrical = any(cs.type == "cylindrical" for cs in countersinks)
    if has_cylindrical:
        return "cylinderhead_hole"
    
    return hole_type


# def detect_holes(shape) -> List[Hole]:
#     """
#     Detect all holes in a shape, including countersinks.
#     Automatically classifies holes as countersunk_hole or cylinderhead_hole.
#     Prevents countersink faces from being detected as separate holes.
#     """
#     holes = []
#     excluded_faces = set()  # Sammle alle Flächen, die als Senkungen identifiziert wurden

#     # Erster Durchlauf: Erkenne alle Bohrungen und ihre Senkungen
#     potential_holes = []
#     for face in shape.Faces:
#         if is_cylindrical_face(face) and is_hole_feature(face, shape):
#             # Get cylinder information
#             radius, center, axis = get_cylinder_info(face)
#             diameter = radius * 2

#             # Classify the hole (basic classification)
#             hole_type = classify_hole(face, shape)

#             # Get depth
#             depth = get_hole_depth(face, shape, hole_type)

#             # Detect countersinks at hole ends
#             countersinks, countersink_face_ids = detect_countersink(face, shape)
            
#             # Sammle alle Senkungsflächen
#             excluded_faces.update(countersink_face_ids)

#             # Re-classify based on countersinks
#             final_hole_type = classify_hole_with_countersink(
#                 face, shape, countersinks, hole_type
#             )

#             potential_holes.append({
#                 'face': face,
#                 'hole_type': final_hole_type,
#                 'diameter': diameter,
#                 'depth': depth,
#                 'center': center,
#                 'axis': axis,
#                 'countersinks': countersinks
#             })

#     # Zweiter Durchlauf: Filtere Bohrungen, deren Flächen als Senkungen markiert sind
#     for hole_data in potential_holes:
#         face_id = id(hole_data['face'])
        
#         # Überspringe diese Fläche, wenn sie als Senkung einer anderen Bohrung identifiziert wurde
#         if face_id in excluded_faces:
#             continue
        
#         # Create hole object with countersinks
#         hole = Hole(
#             hole_type=hole_data['hole_type'],
#             diameter=hole_data['diameter'],
#             depth=hole_data['depth'],
#             position=hole_data['center'],
#             axis=hole_data['axis'],
#             countersinks=hole_data['countersinks']
#         )

#         holes.append(hole)

#     return holes


def print_hole_info(holes: List[Hole]):
    """Hilfsfunktion zum Ausgeben der erkannten Löcher"""
    print(f"\n{'='*50}")
    print(f"Found {len(holes)} hole(s)")
    print(f"{'='*50}")
    
    for i, hole in enumerate(holes, 1):
        print(f"\n--- Hole {i} ---")
        print(f"Type: {hole.hole_type}")
        print(f"Diameter: {hole.diameter:.2f} mm")
        print(f"Depth: {hole.depth:.2f} mm")
        print(f"Position: {hole.position}")
        print(f"Axis: {hole.axis}")
        
        if hole.countersinks:
            print(f"Countersinks: {len(hole.countersinks)}")
            for j, cs in enumerate(hole.countersinks, 1):
                print(f"  {j}. {cs}")

def load_step_file(file_path: str) -> FreeCAD.Document:
    """
    Load a STEP file into a FreeCAD document.

    Args:
        file_path: Path to the STEP file

    Returns:
        FreeCAD document containing the imported geometry from the STEP file
    """
    doc = FreeCAD.newDocument("TempDoc")
    Part.insert(file_path, doc.Name)
    return doc


def get_surface_type(face) -> str:
    """
    Determine the type of a surface/face.

    Args:
        face: A FreeCAD face object

    Returns:
        String describing the surface type
    """
    surface = face.Surface
    surface_type = surface.TypeId

    # Map FreeCAD surface types to readable names
    type_mapping = {
        'Part::GeomPlane': 'Plane',
        'Part::GeomCylinder': 'Cylinder',
        'Part::GeomSphere': 'Sphere',
        'Part::GeomCone': 'Cone',
        'Part::GeomToroid': 'Torus',
        'Part::GeomBSplineSurface': 'BSpline Surface',
        'Part::GeomBezierSurface': 'Bezier Surface',
        'Part::GeomSurfaceOfRevolution': 'Surface of Revolution',
        'Part::GeomSurfaceOfExtrusion': 'Surface of Extrusion',
        'Part::GeomOffsetSurface': 'Offset Surface',
        'Part::GeomTrimmedSurface': 'Trimmed Surface',
    }

    return type_mapping.get(surface_type, surface_type)


def analyze_surfaces(doc: FreeCAD.Document) -> Dict[str, int]:
    """
    Analyze all surfaces in a FreeCAD document and count them by type.

    Args:
        doc: FreeCAD document 

    Returns:
        Dictionary mapping surface type names to their counts
    """
    surface_counts = defaultdict(int)

    for obj in doc.Objects:
        if hasattr(obj, 'Shape'):
            shape = obj.Shape
            for face in shape.Faces:
                surface_type = get_surface_type(face)
                surface_counts[surface_type] += 1

    return dict(surface_counts)


def get_detailed_surface_info(doc: FreeCAD.Document) -> List[Dict]:
    """
    Get detailed information about each surface including type and properties.

    Args:
        doc: FreeCAD document containing the geometry

    Returns:
        List of dictionaries containing detailed surface information
    """
    surfaces = []

    for obj in doc.Objects:
        if hasattr(obj, 'Shape'):
            shape = obj.Shape
            for idx, face in enumerate(shape.Faces):
                surface_type = get_surface_type(face)
                surface_info = {
                    'object_name': obj.Name,
                    'face_index': idx,
                    'surface_type': surface_type,
                    'area': face.Area,
                    'center_of_mass': (face.CenterOfMass.x,
                                      face.CenterOfMass.y,
                                      face.CenterOfMass.z)
                }
                surfaces.append(surface_info)

    return surfaces


def analyze_step_file(file_path: str) -> Tuple[Dict[str, int], int]:
    """
    Main function to analyze a STEP file and return surface statistics.

    Args:
        file_path: Path to the STEP file

    Returns:
        Tuple of (surface_type_counts, total_surface_count)
    """
    doc = load_step_file(file_path)
    surface_counts = analyze_surfaces(doc)
    total_surfaces = sum(surface_counts.values())

    FreeCAD.closeDocument(doc.Name)

    return surface_counts, total_surfaces


def analyze_step_file_complete(file_path: str) -> dict:
    """
    Complete analysis of a STEP file including surfaces and machining features.

    Args:
        file_path: Path to the STEP file

    Returns:
        Dictionary containing all analysis results
    """
    from machining_features import analyze_machining_features, get_machining_statistics

    doc = load_step_file(file_path)

    # Analyze surfaces
    surface_counts = analyze_surfaces(doc)
    total_surfaces = sum(surface_counts.values())

    # Analyze machining features
    features = analyze_machining_features(doc)
    machining_stats = get_machining_statistics(features)

    FreeCAD.closeDocument(doc.Name)

    return {
        'surfaces': {
            'counts': surface_counts,
            'total': total_surfaces
        },
        'machining_features': {
            'features': features,
            'statistics': machining_stats
        }
    }


def print_analysis_results(surface_counts: Dict[str, int], total_surfaces: int):
    """
    Pretty print the analysis results.

    Args:
        surface_counts: Dictionary of surface types and their counts
        total_surfaces: Total number of surfaces
    """
    print("=" * 50)
    print("STEP File Surface Analysis Results")
    print("=" * 50)
    print(f"\nTotal surfaces found: {total_surfaces}\n")
    print("Surface Type Distribution:")
    print("-" * 50)

    # Sort by count (descending)
    sorted_surfaces = sorted(surface_counts.items(),
                            key=lambda x: x[1],
                            reverse=True)

    for surface_type, count in sorted_surfaces:
        percentage = (count / total_surfaces) * 100
        print(f"{surface_type:30} {count:5} ({percentage:5.1f}%)")

    print("=" * 50)


def print_complete_analysis(results: dict):
    """
    Pretty print complete analysis results including surfaces and machining features.

    Args:
        results: Dictionary from analyze_step_file_complete
    """
    # Print surface analysis
    print_analysis_results(results['surfaces']['counts'],
                          results['surfaces']['total'])

    print("\n")

    # Print machining features analysis
    from machining_features import print_machining_analysis
    print_machining_analysis(results['machining_features']['features'])






In [5]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [6]:
test_step_file = Path(r"../../data/testpart.step")
doc = load_step_file(test_step_file.as_posix())
holes = detect_holes(doc.Objects[0].Shape)

holes


[Hole(type='through', diameter=12.00, depth=100.00, countersinks=0)Face=<Face object at 0x1067ed290>,
 Hole(type='blind', diameter=20.00, depth=50.00, countersinks=0)Face=<Face object at 0x1067ed150>,
 Hole(type='blind', diameter=30.00, depth=10.00, countersinks=0)Face=<Face object at 0x1067ebe70>,
 Hole(type='through', diameter=6.00, depth=100.00, countersinks=0)Face=<Face object at 0x1067eb360>]

In [7]:
colors = []


for face in doc.Objects[0].Shape.Faces:
    for h in holes:
        if h.face.isEqual(face):
            colors.append([0,0,1])
        else:
            colors.append([1,0,0])